# Create SHETRAN Raster Data
*Ben Smith | 12/12/2025*

This script is designed to take online downloads and reconfigure them into raster layers that can be used to setup SHETRAN models.

Todo:
- Run at 100m, 200m, 500m and 1000m.
Consider the fixes for the catchments that are below sea level (but that may be one for a later script).

### Preamble

In [5]:
import os
import zipfile

import rasterio
from rasterio.merge import merge
import numpy as np

from scipy.ndimage import generic_filter

import geopandas as gpd

import pandas as pd

import rasterio.features
from shapely.geometry import box

root = 'I:/SHETRAN_GB_2021/02_Input_Data/National Data Inputs for SHETRAN UK/'
resolution_output = 100

def write_ascii(
        array: np,
        ascii_ouput_path: str,
        xllcorner: float,
        yllcorner: float,
        cellsize: float,
        ncols: int = None,
        nrows: int = None,
        NODATA_value: int = -9999,
        data_format: str = '%1.1f'):

        if len(array.shape) > 0:
            nrows, ncols = array.shape

        file_head = "\n".join(
            ["ncols         " + str(ncols),
             "nrows         " + str(nrows),
             "xllcorner     " + str(xllcorner),
             "yllcorner     " + str(yllcorner),
             "cellsize      " + str(cellsize),
             "NODATA_value  " + str(NODATA_value)])

        with open(ascii_ouput_path, 'wb') as output_filepath:
            np.savetxt(fname=output_filepath, X=array,
                       delimiter=' ', newline='\n', fmt=data_format, comments="",
                       header=file_head
                       )


def read_ascii_raster(file_path, data_type=int, return_metadata=True, replace_NA=False):
    """
    Read ascii raster into numpy array, optionally returning headers.
    """
    headers = []
    dc = {}
    with open(file_path, 'r') as fh:
        for i in range(6):
            asc_line = fh.readline()
            headers.append(asc_line.rstrip())
            key, val = asc_line.rstrip().split()
            dc[key] = val
    ncols = int(dc['ncols'])
    nrows = int(dc['nrows'])
    xll = float(dc['xllcorner'])
    yll = float(dc['yllcorner'])
    cellsize = float(dc['cellsize'])
    nodata = float(dc['NODATA_value'])

    arr = np.loadtxt(file_path, dtype=data_type, skiprows=6)
    if replace_NA:
       arr[arr==nodata] = np.nan

    headers = '\n'.join(headers)
    headers = headers.rstrip()

    if return_metadata:
        return arr, ncols, nrows, xll, yll, cellsize, nodata, headers, dc
    else:
        return arr

# Function for cell aggregation
def cell_reduce(array, block_size, func=np.nanmean):
    """
    Resample a NumPy array by reducing its resolution using block aggregation.
    Parameters:
    - array: Input NumPy array.
    - block_size: Factor by which to reduce the resolution.
    - func: Aggregation function (e.g., np.nanmean, np.nanmin, np.nanmax).
            Recomended to use nanmean etc. else you will lose coverage
    """
    shape = (array.shape[0] // block_size, block_size, array.shape[1] // block_size, block_size,)

    return func(array.reshape(shape), axis=(1, 3), )

# Define a function to calculate the mean of valid neighbors:
def fill_holes(values):
    # This will fill all holes with a value in a neighboring cell.

    center = values[4]  # Center pixel in the 3x3 window
    if np.isnan(center):  # If the center is a hole
        neighbors = values[np.arange(len(values)) != 4]  # Exclude the center
        valid_neighbors = neighbors[~np.isnan(neighbors)]  # Keep valid neighbors
        if len(valid_neighbors) > 0:  # Fill only if there are valid neighbors
            return valid_neighbors.mean()
    return center  # Return the original value if not a hole

## Elevation Data

Elevation data for the DEM and minDEM is taken from the OS Terrain 50 dataset. This is free to download:
https://osdatahub.os.uk/downloads/open/Terrain50

Around the coastline, the OS data shows the sea using negative values (presumably taken from a low resolution bathymetry map). It is presumed that this will not impact SHETRAN elevations going forward as the setups do not run to the coast. If much larger negative values were used (i.e. -9999) then this may have a greater impact on those coastal cells compared to the current OS values (0 to -2m or so); although these would still be unlikely to be included within the model domains.

This is used to create the DEM and minimum DEM (which is used for rivers).

OSNI 50m data for Northern Ireland was downloaded as a csv of points. These were converted into an ascii grid using QGIS:
 1. Reprojected from ING to BNG.
2. Converted from points to gridded raster with extents rounded to the appropriate 50m.
3. No data cells (where there were no points in a raster cell) were filled using Fill No Data, ensuring to only look 1 cell away for a value. This does fill some water cells that should be missing data, but this is non-consequential.
4. This filling process was repeated a few times to fill in gaps in the dats where there are lakes etc. Again, non-consequential.
5. Data written as an ascii grid for incorporation into the rasters below. You can use QGIS's _Convert Format_ with _Additional command line parameters_ '-co DECIMAL_PRECISION=1' to write this with 1 decimal place to reduce file size.
6. The NI data would not immediately merge with the GB data due to an issue with the projection. These were very similar (see below), and so I simply copied a GB projection from a prj file to the NI prj file... I don't think this makes any tangible difference.

GB Projection:
<code>
PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0],UNIT["Degree",0.017453292519943295]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000],PARAMETER["False_Northing",-100000],PARAMETER["Central_Meridian",-2],PARAMETER["Scale_Factor",0.999601272],PARAMETER["Latitude_Of_Origin",49],UNIT["Meter",1]]
</code>

Original NI Projection
<code>
PROJCS["British_National_Grid",GEOGCS["GCS_OSGB_1936",DATUM["D_OSGB_1936",SPHEROID["Airy_1830",6377563.396,299.3249646]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]],PROJECTION["Transverse_Mercator"],PARAMETER["False_Easting",400000.0],PARAMETER["False_Northing",-100000.0],PARAMETER["Central_Meridian",-2.0],PARAMETER["Scale_Factor",0.9996012717],PARAMETER["Latitude_Of_Origin",49.0],UNIT["Meter",1.0]]
</code>



In [2]:
# The data is within sub-folders, list these:
OS50_zip_path = os.path.join(root, "terr50_gagg_gb/data/")
OS50_zip_folders = os.listdir(OS50_zip_path)
OS50_zip_folders = [a for a in OS50_zip_folders if 'Unzipped_data' not in a]

# Setup a new folder to hold the unzipped data:
OS50_unzipped_folder = os.path.join(OS50_zip_path, 'Unzipped_data/')
if not os.path.exists(OS50_unzipped_folder):
    os.mkdir(OS50_unzipped_folder)

# Unzip the data:
for OS50_zip_folder in OS50_zip_folders:
    zip_folders = os.listdir(os.path.join(OS50_zip_path, OS50_zip_folder))
    for zip_folder in zip_folders:
        print(os.path.join(OS50_zip_path, OS50_zip_folder, zip_folder))
        with zipfile.ZipFile(os.path.join(OS50_zip_path, OS50_zip_folder, zip_folder), 'r') as zip_ref:
            zip_ref.extractall(OS50_unzipped_folder)

Join the elevation rasters into a single file.

In [ ]:
# List all .asc files in the folder
asc_files = [os.path.join(OS50_unzipped_folder, f) for f in os.listdir(OS50_unzipped_folder) if f.endswith('.asc')]

# Open the GB files using rasterio:
count = 1
raster_list = []
for asc_file in asc_files:
    print(count, "/", len(asc_files))
    raster = rasterio.open(asc_file,)
    raster_list.append(raster)
    count += 1

# ---

# Open the filled NI file using rasterio:
print('NI', "/", len(asc_files))
raster = rasterio.open(os.path.join(root, 'OSNI_OpenData_50m/OSNI_OpenData_50m_BNG_Filled.asc'),)
raster_list.append(raster)

# ---

# Combine (merge) the rasters:
merged_raster, merged_transform = merge(raster_list, nodata=-9999)

# Close the opened raster files - you may be able to incorporate this into the loop above.
for raster in raster_list:
    raster.close()

# Extract the first raster band and change 0s to -9999:
merged_raster = merged_raster[0]
# merged_raster[merged_raster == 0] = -9999  # This was changed to merge(..., nodata=-9999) as it created issues in the fens

National_OS50_path = os.path.join(root, 'Processed Data', 'National_OS50.asc')

# Write the file as an ascii:
write_ascii(
    array=merged_raster,
    ascii_ouput_path=National_OS50_path,
    xllcorner=merged_transform[2],
    yllcorner=merged_transform[5]-(merged_raster.shape[0]*merged_transform[0]),
    cellsize=merged_transform[0],
)

Regrid the elevation rasters to the desired size.

Note that this does assume that the lower left corner of the national OS50 file is at 0,0, easting northing. Check this if you are redoing this work. you can load the header of the file using the following code:
<code>
headers = []
with open(OS50_zip_path + 'National_OS50.asc', 'r') as fh:
for i in range(6):
asc_line = fh.readline()
headers.append(asc_line.rstrip())
headers
</code>

The first stage of this is to ensure that the 50m data is of the same extent as the 1km data. Rows and columns are added to ensure this. This means that the data has an extent that is in 1km, so can be resampled to divisions of this (1km, 500m, 200m, 100m). This may not work if you try other resolutions as, because the calculations will run from the top left, not the bottom left, the resampled dataset may not have llx/lly coordinates of 0,0. Think about this if you want to use other resolutions!

In [ ]:
national_OS50, _, _, _, _, _, _, _, OS50_header = read_ascii_raster(National_OS50_path, data_type=float, replace_NA=True)

In [ ]:
# # If you have not loaded in the dataset (perhaps because you are only testing the code), you can check the dimensions of the 50m dataset using this code:
#
# OS50_header = {}
# with open(OS50_zip_path + 'National_OS50.asc', 'r') as fh:
#     for i in range(6):
#         asc_line = fh.readline()
#         key, val = asc_line.rstrip().split()
#         OS50_header[key] = val
# OS50_header

In [ ]:
# Resize the national dataset to match existing SHETRAN inputs:
# Resize the inputs to the desired SHETRAN grid (top right corner should be x: 661000, y: 1241000):
row_difference = ((661*1000) - float(OS50_header['nrows']) * float(OS50_header['cellsize'])) / float(OS50_header['cellsize'])
col_difference = ((1241*1000) - float(OS50_header['ncols']) * float(OS50_header['cellsize'])) / float(OS50_header['cellsize'])

if row_difference > 0:
    # Create the rows of -9999
    new_rows = np.full((row_difference, national_OS50.shape[1]), -9999)
    # Add the new rows to the top
    national_OS50 = np.vstack((new_rows, national_OS50))

# repeat for columns:
if row_difference > 0:
    new_cols = np.full((national_OS50.shape[0], col_difference), -9999)
    national_OS50 = np.hstack((national_OS50, new_cols))  # Remember that these need adding at the end.

_I have removed the code chuck below as I think it is superfluous. There were some issues resulting from changing 0 values to NA when in fact these are valid elevations. This has been corrected and the code designed to fix/fill the holes left below in case of potential future uses.

*_it may be of use in the Northern Ireland catchments, where there is a greater presence of NA values over lakes._*

_This will fill the holes (na/-9999 values) in the dataset - this code will only fill calls that have a valid value in an adjacent cell._

<code>
\# Replace hole_value with NaN for processing
raster[raster == -9999] = np.nan
\# Apply the function iteratively
filled_national_OS50 = generic_filter(national_OS50, fill_holes, size=3, mode='constant', cval=np.nan)
filled_national_OS50[filled_national_OS50 == np.nan] = -9999
\# Write the file as an ascii:
write_ascii(
    array=filled_national_OS50,
    ascii_ouput_path=f'{OS50_zip_path}National_OS50_DEM_preprocessed.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=float(OS50_header['cellsize'])
)
</code>

**The following code will give warnings when trying to take the mean of cells that are all np.nan - don't worry, this is doing what it should. (Probably everything in QGIS or similar at the end though).**

In [ ]:
# Define the block size for aggregation
resolution_input = float(OS50_header['cellsize'])
block_size = int(resolution_output/resolution_input)  # For 50m -> 100m, use a block size of 2

# Resample using the mean and minimum:
DEM = cell_reduce(national_OS50, block_size, np.mean)
minDEM = cell_reduce(national_OS50, block_size, np.min)

# -9999 was converted to np.nan in the loading phase, convert it back
DEM[np.isnan(DEM)] = -9999
minDEM[np.isnan(minDEM)] = -9999

In [3]:
# Write the file as an ascii:
write_ascii(
    array=DEM,
    ascii_ouput_path=f'{root}/Processed Data/National_OS50_DEM_{resolution_output}m.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=resolution_output
)

# Write the file as an ascii:
write_ascii(
    array=minDEM,
    ascii_ouput_path=f'{root}/Processed Data/National_OS50_minDEM_{resolution_output}m.asc',
    xllcorner=OS50_header['xllcorner'],
    yllcorner=OS50_header['yllcorner'],
    cellsize=resolution_output
)


## Land Use Datasets

These are available as 25m and 1km rasters or as vector layers. Vectors are prefered as these allow for greater precision when building lower resolution rasters.

All data is CEH Land Use data (2007), available online for GB and NI (separately):
_https://catalogue.ceh.ac.uk/documents/e02b4228-fdcf-4ab7-8d9d-d3a16441e23d_

Processing steps:
1. Download data (manually, unzip if necessary).
2. Merge data classes as per the table below and save the updated shapefiles.
a. Load each regional shapefile in turn.
b. Remove the unnecessary data.
c. Dissolve the polygons to reduce the files size.
d. Write the shapefiles (these can be removed once the rest of these steps are completed).
3. Merge the shapefiles into a single UK wide vector dataset.
4. Read the UK dataset and resample into the desired resolution and write as asc files. This has the following steps:
a. Create a vector grid of the desired resolution covering the standard SHETRAN UK domain and give each cell an ID.
b. Intersect this grid with the UK land cover data so that each polygon is within a single cell boundary and has the grid ID that it is within.
c. Calculate the area of each intersected polygon, filter using the area and grid cell ID, and remove duplicates, leaving only a single polygon per grid cell (the one with the largest area).
d. Join these polygons back to the original grid (so that the data can be displayed as a regular grid, rather than 1 irregular polygon per grid cell).
e. Rasterise and save the data

|	**LCM2007 Class**	|	**LCM2007 Class Number**	|	**SHETRAN Class**	|	**SHETRAN Class Number**	|
|----|----|----|----|
|	Broadleaved woodland	|	1	|	DeciduousForest	|	4	|
|	Coniferous Woodland	|	2	|	EvergreenForest	|	5	|
|	Arable and Horticulture	|	3	|	Arable	|	1	|
|	Improved Grassland	|	4	|	Grass	|	3	|
|	Rough grassland	|	5	|	Grass	|	3	|
|	Neutral Grassland	|	6	|	Grass	|	3	|
|	Calcareous Grassland	|	7	|	Grass	|	3	|
|	Acid Grassland	|	8	|	Grass	|	3	|
|	Fen, Marsh and Swamp	|	9	|	Shrub	|	6	|
|	Heather	|	10	|	Shrub	|	6	|
|	Heather grassland	|	11	|	Shrub	|	6	|
|	Bog	|	12	|	Shrub	|	6	|
|	Montane Habitats	|	13	|	Shrub	|	6	|
|	Inland Rock	|	14	|	BareGround	|	2	|
|	Saltwater	|	15	|	BareGround	|	2	|
|	Freshwater	|	16	|	BareGround	|	2	|
|	Supra-littoral Rock	|	17	|	BareGround	|	2	|
|	Supra-littoral Sediment	|	18	|	BareGround	|	2	|
|	Littoral Rock	|	19	|	BareGround	|	2	|
|	Littoral sediment	|	20	|	BareGround	|	2	|
|	Saltmarsh	|	21	|	BareGround	|	2	|
|	Urban	|	22	|	Urban	|	7	|
|	Suburban	|	23	|	Urban	|	7	|

#### Step 2

In [10]:
# Define the reclassification dictionary
reclass_dict = {  # (CEH LCM to SHETRAN Classes)
    1: 4, 2: 5, 3: 1,
    4: 3, 5: 3, 6: 3, 7: 3,8: 3,
    9: 6, 10: 6, 11: 6, 12: 6, 13: 6,
    14: 2, 15: 2, 16: 2,  17: 2,  18: 2,  19: 2, 20: 2, 21: 2,
    22: 7, 23: 7
}

# List the shapefiles in GB:
GB_LCM  = os.path.join(root, 'Land Use Inputs/LCM_2007_vector_GB_Digimap/lcm-2007-vec_5779248')
GB_LCM_files = os.listdir(GB_LCM)
shapefiles = [os.path.join(GB_LCM, sf) for sf in GB_LCM_files if sf.endswith('.shp')]

NI_LCM  = os.path.join(root, 'Land Use Inputs/LCM_2007_vector_NI_Digimap/lcm-2007-vec-ni_4578539')
NI_LCM_files = os.listdir(NI_LCM)
shapefiles.append([os.path.join(NI_LCM, sf) for sf in NI_LCM_files if sf.endswith('.shp')])

# Run through the files (including NI):
counter = 1
for shapefile in shapefiles:
    print(counter, '/', len(shapefiles))

    # Read in the data:
    sf = gpd.read_file(shapefile)

    # Reclassify from LCM to SHETRAN classes'
    sf['SHETRAN'] = sf['INTCODE'].map(reclass_dict)

    # Reproject the Northern Ireland file into BNG (from ING):
    if 'LCM_2007_vector_NI_Digimap' in shapefile:
        sf = sf.to_crs(epsg=27700)

    # Cull the columns you don't need:
    columns = sf.columns
    columns = [column for column in columns if column not in ['SHETRAN', 'geometry']]
    sf.drop(columns, inplace=True, axis=1)

    # Dissolve the polygons to reduce file size:
    sf_dissolved = sf.dissolve('SHETRAN')

    # Save the updated shapefile:
    sf_dissolved.to_file(
        os.path.join(root, "Land Use Inputs/Reclassified shapefiles", os.path.basename(shapefile))
    )

    counter += 1

The projection files again don't quite match so I have manually copied the projection from the GB files to the NI file... This is a very small difference and does not seem to make any difference to the polygon locations.

Original: PARAMETER["Scale_Factor",0.9996012717]

Updated: PARAMETER["Scale_Factor",0.999601272]


#### Step 3

In [ ]:
# List the shapefiles in GB:
shapefile_path = os.path.join(root, 'Land Use Inputs/Reclassified shapefiles')
shapefiles = os.listdir(shapefile_path)
shapefiles = [os.path.join(shapefile_path, sf) for sf in shapefiles if sf.endswith('.shp')]

# Merge into a single file:
gdfs = []
for shapefile in shapefiles:
    gdfs.append(gpd.read_file(shapefile))

# Merge all GeoDataFrames into one
merged_gdf = gpd.GeoDataFrame(pd.concat(gdfs, ignore_index=True))

# Save the merged GeoDataFrame to a new shapefile
merged_gdf.to_file(shapefile_path + '/LCM_2007_vector_UK_BNG.shp')

#### Step 4
**Regrid the data into the desired raster files:**
This next code chunk generates the gridded data of the desired resolution and so should be run multiple times for each of the desired resolutions.

In [1]:
# Load the vector data (merged shapefile)
gdf = gpd.read_file(shapefile_path + '/LCM_2007_vector_UK_BNG.shp')

# Step 2: Create a vector grid
xmin, ymin, xmax, ymax = 0, 0, 661000, 1241000  # British National Grid boundaries
cell_size = resolution_output  # 100m resolution
cols = np.arange(xmin, xmax, cell_size)
rows = np.arange(ymin, ymax, cell_size)

grid_cells = []
for x in cols:
    for y in rows:
        grid_cells.append(box(x, y, x + cell_size, y + cell_size))

# Turn this into a geodataframe and give it an ID
grid = gpd.GeoDataFrame({"geometry": grid_cells}, crs=gdf.crs)
grid['ID'] = np.arange(0, grid.shape[0])

# Step 1: Intersect the grid and the shapefile
intersected = gpd.overlay(grid, gdf, how='intersection', keep_geom_type=False)

# Step 2: Calculate the area of each intersected polygon
intersected["area"] = intersected.area

# Step 3: Sort the intersected DataFrame by 'ID' and 'area' and crop to only the largest land type per cell:
intersected_sorted = intersected.sort_values(by=["ID", "area"], ascending=[True, False])

# Step 4: Drop duplicates based on 'ID', keeping only the largest area
filtered_intersected = intersected_sorted.drop_duplicates(subset="ID")
# filtered_intersected.to_file(shapefile_path + '/filtered_intersected.shp')

# 5. Converting filtered_intersected straight to raster misses cells, instead join the LC classes back to the grid:
# Perform the left join on the 'ID' column
grid_with_intersected = grid.merge(filtered_intersected[['SHETRAN', 'ID']], on="ID", how="left", suffixes=('_grid', '_intersected'))
# grid_with_intersected.to_file(shapefile_path + '/grid_with_intersected.shp')

# Step 6: Rasterize the intersected polygons
# Define the raster properties
transform = rasterio.transform.from_bounds(xmin, ymin, xmax, ymax, len(cols), len(rows))

# Prepare shapes and values for rasterisation:
shapes = ((geom, value) for geom, value in zip(grid_with_intersected.geometry, grid_with_intersected['SHETRAN']))

# Rasterize:
raster = rasterio.features.rasterize(
    shapes,
    out_shape=(len(rows), len(cols)),
    transform=transform,
    fill=-9999,  # NoData value
    dtype="int32"
)

# Convert 0s to -9999s for no data values:
raster[raster == 0] = -9999

write_ascii(
    array=raster,
    ascii_ouput_path=f'{root}/Processed Data/CEH_LCM_2007 {resolution_output}m.asc',
    xllcorner=xmin,
    yllcorner=ymin,
    cellsize=cell_size,
    NODATA_value=-9999,
    data_format='%1.0f'
)